# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
df = pd.read_csv("weather_data.csv")
df.head()

,city,lat,lng,max_temp,humidity,wind_speed,cloudiness,date,country
0,Paamiut,61.99,-49.67,41.540,85,3.41,100,1602967162,GL
1,Rikitea,-23.12,-134.97,74.228,79,9.54,100,1602967162,PF
2,Tiksi,71.69,128.87,18.950,97,5.07,100,1602967163,RU
3,Vaini,-21.20,-175.20,77.000,94,5.10,75,1602967163,TO
4,Punta Arenas,-53.15,-70.92,44.600,93,6.70,100,1602967163,CL


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(df[["lat", "lng"]], weights=df["humidity"]) 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:

#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.



filtered_df = df[df["max_temp"] < 75]
filtered_df = filtered_df[filtered_df["max_temp"] > 60]
filtered_df = filtered_df[filtered_df["humidity"] < 30]
filtered_df = filtered_df[filtered_df["humidity"] > 20]
filtered_df = filtered_df[filtered_df["wind_speed"] < 10]
hotel_df = filtered_df[filtered_df["cloudiness"] < 75]

hotel_df


,city,lat,lng,max_temp,humidity,wind_speed,cloudiness,date,country
18,Loreto,22.27,-101.97,74.858,26,1.80,0,1602967168,MX
43,Rawson,-43.30,-65.10,72.320,22,8.55,40,1602967174,AR
180,Weiser,44.25,-116.97,73.400,27,3.22,1,1602967212,US
261,Buraidah,26.33,43.98,71.600,26,1.00,0,1602967235,SA
412,Lebanon,36.21,-86.29,66.992,23,2.10,1,1602967276,US
437,Gobabis,-22.45,18.97,71.546,21,5.26,32,1602967284,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
len(hotel_df)

6

In [40]:
lats = hotel_df['lat'].tolist()
lngs = hotel_df['lng'].tolist()

locs = list(zip(*[lats, lngs]))
locs

[(22.27, -101.97),
 (-43.3, -65.1),
 (44.25, -116.97),
 (26.33, 43.98),
 (36.21, -86.29),
 (-22.45, 18.97)]

In [64]:

hotels = []
for i in range(len(hotel_df)):
    coordinates = f"{locs[i][0]}, {locs[i][1]}"
    target_search = "hotel"
    radius = 5000

    # set up a parameters dictionary
    params = {
        "location": coordinates,
        "keyword": target_search,
        "radius": radius,
#        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code > 399:
        print(response.status_code)
        break
    
    hotels.append(response.json())
    

In [68]:
print(hotels[0]['results'][0].keys())

hotels[0]['results'][0]['name']

dict_keys(['business_status', 'geometry', 'icon', 'name', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'])


'Miguel Angel Hotel'

In [73]:
names = []
for j in range(len(hotels)):
    names.append(hotels[j]['results'][0]['name'])
    

hotel_df["Hotel Name"] = names
hotel_df

,city,lat,lng,max_temp,humidity,wind_speed,cloudiness,date,country,Hotel Name
18,Loreto,22.27,-101.97,74.858,26,1.80,0,1602967168,MX,Miguel Angel Hotel
43,Rawson,-43.30,-65.10,72.320,22,8.55,40,1602967174,AR,Hotel Punta León
180,Weiser,44.25,-116.97,73.400,27,3.22,1,1602967212,US,State Street Motel
261,Buraidah,26.33,43.98,71.600,26,1.00,0,1602967235,SA,فندق براند القصيم
412,Lebanon,36.21,-86.29,66.992,23,2.10,1,1602967276,US,Avid Hotels Nashville - Lebanon
437,Gobabis,-22.45,18.97,71.546,21,5.26,32,1602967284,NaN,Goba Lodge


In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [77]:

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))